In [1]:
!pip install -U tensorflow==2.3.0

     |████████████████████████████████| 320.4 MB 34 kB/s s eta 0:00:01   |█▊                              | 17.7 MB 8.7 MB/s eta 0:00:35     |██████████████████████▏         | 222.4 MB 37.4 MB/s eta 0:00:03     |██████████████████████████▊     | 267.2 MB 24.0 MB/s eta 0:00:03
     |████████████████████████████████| 26.1 MB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 20.1 MB 51.1 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 48.0 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.4
    Uninstalling scipy-1.5.4:
      Successfully uninstalled scipy-1.5.4
  Attem

In [2]:
!pip install tensorflow-quantum

     |████████████████████████████████| 5.9 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 22.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 17.3 MB/s eta 0:00:01     |███████████                     | 1.9 MB 17.3 MB/s eta 0:00:01     |█████████████████████           | 3.7 MB 17.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 55 kB/s s eta 0:00:01     |████████████████████▍           | 808 kB 38.2 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.14.0
    Uninstalling protobuf-3.14.0:
      Successfully uninstalled protobuf-3.14.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.7.1
    Uninstalling sympy-1.7.1:
      Successfully uninstalled sympy-1.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ortools 8.1.8487 requires absl-py>=

In [3]:
#Importing tensorflow
import tensorflow as tf
import tensorflow_quantum as tfq

from tensorflow.keras import datasets, layers, models

#Importing some tensorflow quantum stuff
import cirq
import sympy

#For data manipulation
import numpy as np

import collections

#to plot stuff
import matplotlib.pyplot as plt

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array, smart_resize
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.preprocessing import image

In [128]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
class_name = df['labels'].value_counts().index
df['labels'] = df['labels'].astype('category')
df['label_num'] = df['labels'].cat.codes

In [177]:
IMAGE_SIZE = (90, 90)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./225,
                                                                validation_split=0.4)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [178]:
train_dir = '../input/plant-pathology-2021-fgvc8/train_images'

In [179]:
train_generator = train_datagen.flow_from_dataframe(dataframe=df,
                                                    directory=train_dir,
                                                    subset='training',
                                                    x_col="image",
                                                    y_col="label_num",
                                                    shuffle=True,
                                                    color_mode="grayscale",
                                                    target_size=IMAGE_SIZE,
                                                    batch_size=11180,
                                                    class_mode='raw')

# val_generator = train_datagen.flow_from_dataframe(dataframe=df,
#                                                     directory=train_dir,
#                                                     subset="validation",
#                                                     x_col="image",
#                                                     y_col="label_num",
#                                                     shuffle=True,
#                                                     color_mode="grayscale",
#                                                     target_size=IMAGE_SIZE,
#                                                     batch_size=10,
#                                                     class_mode='raw')

Found 11180 validated image filenames.


In [180]:
# for n,x in enumerate(train_generator):
#     print(type(x[0]), type(x[1]))
#     break

KeyboardInterrupt: 

In [181]:
train_x = train_generator[0][0]
train_y = train_generator[0][1]

In [145]:
# train_x = np.ndarray((11180, 150, 150,1))
# train_y = np.ndarray((11180))

In [152]:
# for x in train_generator:
#     np.append(train_x, x[0])
#     np.append(train_y, x[1])
#     break

In [153]:
# train_y

array([6.90839482e-310, 6.90839482e-310, 4.67986397e-310, ...,
       8.09028184e-003, 4.75694704e-003, 5.31250331e-003])

In [154]:
# train_x.shape,train_y.shape

((11180, 150, 150, 1), (11180,))

In [182]:
#Defining the whole thing!!!
class QConv(tf.keras.layers.Layer):
  #initializaiton of the QCNN layer
  def __init__(self, filter_size, depth, activation = None, name = None, kernel_regularizer=None, **kwangs):
    #Standard notation thingy
    super(QConv, self).__init__(name=name, **kwangs)

    #Defining of the varaibles
    self.filter_size = filter_size
    self.depth = depth
    self.learning_params = []
    self.QCNN_layer_gen()
    self.activation = tf.keras.layers.Activation(activation)
    self.kernel_regularizer = kernel_regularizer
  
  #Initialize parameters for the quantum gates
  def _get_new_param(self):
    #Literally just generates a string "p0"... Instead of 0 it's just a number that
    new_param = sympy.symbols('p'+str(len(self.learning_params)))
    #Increase the size of the list (thus the numbers keep increasing (so there's no duplicates))
    self.learning_params.append(new_param)
    return new_param
  
  #This just defines 2 parameterized qubit gates and places them
  def _QConv(self, step, target, qubits):
    #First defining a Z and an X gate. First part = the rotation value (where we place our parameter), second part = where we place our gates
    yield cirq.CZPowGate(exponent=self._get_new_param())(qubits[target], qubits[target+step])
    yield cirq.CXPowGate(exponent=self._get_new_param())(qubits[target], qubits[target+step])
  
  def QCNN_layer_gen(self):
    #Pixels = the area which the filter will cover
    pixels = self.filter_size**2

    #So we're going to take our kernal and map it to qubits
    cirq_qubits = cirq.GridQubit.rect(self.filter_size, self.filter_size)

    #How you define the start of a quantum circuit
    input_circuit = cirq.Circuit()

    #There's another set of parameterized gates here. And we've got to define it's parameters
    input_params = [sympy.symbols('a%d' %i) for i in range(pixels)]

    #Now we apply those initial RX gates at the beginning for each qubit
    for i, qubit in enumerate(cirq_qubits):
      input_circuit.append(cirq.rx(np.pi*input_params[i])(qubit))
    
    #We're going to start antoher part, this time it's the kernal part
    QCNN_circuit = cirq.Circuit()

    #Basically something to help with the architechture of the kernal part (to help with the placement of the X and Z gates)
    step_size = [2**i for i in range(np.log2(pixels).astype(np.int32))]
    
    #This is the appending of said X and Z gates
    for step in step_size:
      for target in range(0, pixels, 2*step):
        QCNN_circuit.append(self._QConv(step,target,cirq_qubits))
    
    #now take the 2 parts of the quantum circuit to merge them all together
    full_circuit = cirq.Circuit()
    full_circuit.append(input_circuit)
    full_circuit.append(QCNN_circuit)

    #save it to use it later
    self.circuit = full_circuit

    #Save the parameters to use later
    self.params = input_params + self.learning_params

    #Save the operators (for the output) for later use
    self.op = cirq.Z(cirq_qubits[0])
  
  #Intializes everything... It creates the layer (with weights and stuff)
  def build(self, input_shape):
    #What's the input (image) width? Height? Number of channels?
    self.width = input_shape[1]
    self.height = input_shape[2]
    self.channel = input_shape[3]

    # The number of times which the kernal will pass on the image
    self.num_x = self.width - self.filter_size + 1
    self.num_y = self.height - self.filter_size + 1

    #Initializing the kernal! name, (how many (if there are 8, then it'll be a rectangular prism, but ostensibly 8 different kernals), channels, number of parameters each)
    #Then we initialzie the parameters, plus slap on a regularator if we wanted to
    self.kernel = self.add_weight(name = 'kernal',
                                 shape = [self.depth, self.channel, len(self.learning_params)],
                                 initializer = tf.keras.initializers.glorot_normal(),
                                 regularizer = self.kernel_regularizer)
    
    #We take our thing and convert it to a (quantum?) tensor
    self.circuit_tensor = tfq.convert_to_tensor([self.circuit] * self.num_x * self.num_y * self.channel)
  
  #Where the computation happens
  def call(self, inputs):
    #This is generating a giant stack of all the segements of the inputs which we're going to pass over the kernal
    #Also: It's just adding the slice to the whole stack each time. (It works like. Which cord on the map, then take a bit out of that with the size) 
    stack_set = None
    for i in range(self.num_x):
      for j in range(self.num_y):
        slice_part = tf.slice(inputs, [0, i, j, 0], [-1, self.filter_size, self.filter_size, -1])
        slice_part = tf.reshape(slice_part, shape=[-1, 1, self.filter_size, self.filter_size, self.channel])
        if stack_set == None:
          stack_set = slice_part
        else:
          stack_set = tf.concat([stack_set, slice_part], 1)
    #Then we just reformat it
    stack_set = tf.transpose(stack_set, perm=[0, 1, 4, 2, 3])
    stack_set = tf.reshape(stack_set, shape=[-1, self.filter_size**2])

    #Kind of reformats (except with some duplication) the (quantum?) tensor into a usable form
    circuit_inputs = tf.tile([self.circuit_tensor], [tf.shape(inputs)[0], 1])
    circuit_inputs = tf.reshape(circuit_inputs, shape=[-1])
    tf.fill([tf.shape(inputs)[0]*self.num_x*self.num_y, 1], 1)
    
    #Gonna take our inputs (now in the form of the stack) and pass them through our kernals
    outputs = []
    for i in range(self.depth):
      #Now we call the kernals we defined in build
      controller = tf.tile(self.kernel[i], [tf.shape(inputs)[0]*self.num_x*self.num_y, 1])
      #Actually passing into the QCNN layer
      outputs.append(self.single_depth_QCNN(stack_set, controller, circuit_inputs))
    #reformating
    output_tensor = tf.stack(outputs, axis=3)
    output_tensor = tf.math.acos(tf.clip_by_value(output_tensor, -1+1e-5, 1-1e-5)) / np.pi
    
    #Take our output, shove it through the activation, and then return it
    return self.activation(output_tensor)
  
  def single_depth_QCNN(self, input_data, controller, circuit_inputs):
    #Shove it on GPU
    with tf.device('/device:GPU:0'):
      #Reformat the input data
      input_data = tf.concat([input_data, controller],1)
      #Then taking our input and shoving it through the QCNN (along with it's paramters)
      QCNN_output = tfq.layers.Expectation()(circuit_inputs,
                                            symbol_names = self.params,
                                            symbol_values = input_data,
                                            operators = self.op)
      #Reformat x 2
      QCNN_output = tf.reshape(QCNN_output, shape=[-1, self.num_x, self.num_y, self.channel])
      return tf.math.reduce_sum(QCNN_output, 3)

In [183]:
#Defining the varaibles to intitalize the QCNN
width = 90
height = 90

In [184]:
#Initializing the model
qcnn_model = models.Sequential()
#The QCNN!!!!!
qcnn_model.add(QConv(filter_size=2, depth=4, activation='relu', 
                     name='qconv1', input_shape=(width, height, 1)))
#So we can pase it to linear layers
qcnn_model.add(layers.Flatten())
qcnn_model.add(layers.Dense(32, activation='relu'))
#Important to have a softmax here... So all the probabilities add up to 100%
qcnn_model.add(layers.Dense(12, activation='softmax'))

In [185]:
qcnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
qconv1 (QConv)               (None, 89, 89, 4)         24        
_________________________________________________________________
flatten (Flatten)            (None, 31684)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1013920   
_________________________________________________________________
dense_1 (Dense)              (None, 12)                396       
Total params: 1,014,340
Trainable params: 1,014,340
Non-trainable params: 0
_________________________________________________________________


In [186]:
qcnn_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [191]:
qcnn_history = qcnn_model.fit(train_x, train_y, steps_per_epoch=500,
                        epochs=10, batch_size=5)

Epoch 1/10


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [193]:
train_y.shape, train_x.shape

((11180,), (11180, 90, 90, 1))

In [ ]:
tf.config.run_functions_eagerly(True)